# Init

In [ ]:
import common

common.GrConf() #build common scope config
productsCaller = common.GrGetAPI('Objects','products')
productsCaller.buildParam("active=1")

# Workflow to import Grocy product data
Init gets everything ready for product calls. It'll call it next

In [ ]:
productsCaller.get()
r=productsCaller.r
if not "200" in str(r):
    print(f'Call failed :( Error:{str(r)}')
else:
    print('Success')

# Save offline copy (limited to success)

In [ ]:
if "200" in str(r):
    resp = input('Confirm save offline copy?')
    if "y" in resp or "Y" in resp:        
        if os.path.exists("localgrocyproducts.json"):
            record=open("localgrocyproducts.json",mode="w")
        else:
            record=open("localgrocyproducts.json",mode="x")

        json.dump(r.json(),record)
        record.close()
        print("localgrocyproducts.json was saved")

# Mapping
Loop through the relevant interfaces that provide names to IDs
Todo: adapt this to also save the mapping locally for building mass uploads if needed

In [ ]:
callThese = ['locations','shopping_locations','product_groups','quantity_units']
columnMap = {'locations':['location_id','default_consume_location_id'],
             'shopping_locations':['shopping_location_id'],
             'product_groups':['product_group_id'],
             'quantity_units':['qu_id_purchase','qu_id_stock','qu_id_consume','qu_id_price']}
rjson = r.json()
for a in callThese:

    iterCaller = common.GrGetAPI('Objects',a)
    iterCaller.buildParam("active=1")
    iterCaller.get()
    if "200" not in str(iterCaller.r):
        print(r+' API failure')


# Join the API calls so far into one rjson master file

    for x in rjson:
        for y in iterCaller.r.json():
                for z in range(len(columnMap[a])):
                    #print(z)
                    #print(columnMap[a])
                    #print(columnMap[a][z])
                    #print(x['location_id'])
                    if x[columnMap[a][z]] == y['id']:
                        x[columnMap[a][z]] = y['name']
                        
# Hard type certain problematic inferences
# this probably ought to be a main function

In [ ]:
# Ex data structures I was using before a DDIC
#callThese = ['locations','shopping_locations','product_groups','quantity_units']
#columnMap = {'locations':['location_id','default_consume_location_id'],
#             'shopping_locations':['shopping_location_id'],
#             'product_groups':['product_group_id'],
#             'quantity_units':['qu_id_purchase','qu_id_stock','qu_id_consume','qu_id_price']}'''
rjson = r.json()
callThese = []
columnMap = {}
alias=common.CONF.build_alias(rjson)


# Tabulate Data
Input response to DataFrame & Display as an ITable

In [ ]:
import pandas as pd
import io
from itables import init_notebook_mode
from itables import show

rjson = r.json()
callThese = []
columnMap = {}
alias=common.CONF.build_alias(rjson)

init_notebook_mode(all_interactive=True)
productsFrame = pd.DataFrame(rjson)
for x in alias:
    #This typing probably goes away once I make hard typing come in
    productsFrame[x] = productsFrame[x].astype('Int64')
    productsFrame[x] = productsFrame[x].astype('str')
    for y in alias[x]:
        for z in y:
            productsFrame.loc[productsFrame[x] == str(z), x] = str(y[z])
show(productsFrame,buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
# filter by query example
# use user input for column and value
show(productsFrame.loc[productsFrame['product_group_id'] == 'Staples'],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
#filter by substring example
#use user input for column and value
show(productsFrame[productsFrame['name'].str.contains('ANY')],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

# Load to mass load spreadsheet
Make any filterations in place in productsFrame, then call the next block to load the list into a mass put spreadsheet. Then call the mass put script